In [1]:
from typing import Dict, List, Tuple
import random
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import gym
import envs
from datetime import datetime
import json


import math
from pyfmi import load_fmu
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import os
import pandas as pd


%load_ext autoreload
%autoreload 2

In [2]:
##tair = []
##action = []
##pmv = []
##qheat = []
##reward = []
##occ = []
##iterations = 10000
##
##for i in range(iterations):
##    action_ = random.randint(0,env.action_dim -1 )
##    obs, reward_, done, info = env.step(action=action_)
##    reward.append(reward_)
##    action.append(env.action_to_temp[action_])
##    pmv.append(info['pmv'][0])
##    d = env.observation_to_dict(obs)
##    tair.append(d["tair_in"][0])
##    qheat.append(d["qheat_in"][0])
##    occ.append(d["occ_in"][0])
##
##
##tair = np.array(tair)
##action = np.array(action)
##pmv = np.array(pmv)
##qheat = np.array(qheat)
##reward = np.array(reward)
##occ = np.array(occ)
##
##
##t = np.linspace(0.0, iterations -1, iterations)
##
### Plotting the summary of simulation
##fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.04,
##                    specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}],
##                           [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": False}]])
### Add traces
##fig.add_trace(go.Scatter(name='Tair(state)', x=t, y=tair.flatten(), mode='lines', line=dict(width=1, color='cyan')),
##              row=1, col=1)
##fig.add_trace(go.Scatter(name='Tair_avg', x=t, y=pd.Series(tair.flatten()).rolling(window=24).mean(), mode='lines',
##              line=dict(width=2, color='blue')), row=1, col=1)
##fig.add_trace(go.Scatter(name='Tset(action)', x=t, y=action.flatten(), mode='lines', line=dict(width=1, color='fuchsia')),
##              row=2, col=1)
##fig.add_trace(go.Scatter(name='Tset_avg', x=t, y=pd.Series(action.flatten()).rolling(window=24).mean(), mode='lines',
##              line=dict(width=2, color='purple')), row=2, col=1)
##fig.add_trace(go.Scatter(name='Pmv', x=t, y=pmv.flatten(), mode='lines', line=dict(width=1, color='gold')),
##              row=3, col=1)
##fig.add_trace(go.Scatter(name='Pmv_avg', x=t, y=pd.Series(pmv.flatten()).rolling(window=24).mean(), mode='lines',
##              line=dict(width=2, color='darkorange')), row=3, col=1)
##fig.add_trace(go.Scatter(name='Heating', x=t, y=qheat.flatten(), mode='lines', line=dict(width=1, color='red')),
##              row=4, col=1, secondary_y=False)
##fig.add_trace(go.Scatter(name='Heating_cumulative', x=t, y=np.cumsum(qheat.flatten()), mode='lines',
##              line=dict(width=2, color='darkred')), row=4, col=1, secondary_y=True)
##fig.add_trace(go.Scatter(name='Reward', x=t, y=reward.flatten(), mode='lines', line=dict(width=1, color='lime')),
##              row=5, col=1, secondary_y=False)
##fig.add_trace(go.Scatter(name='Reward_cum', x=t, y=np.cumsum(reward.flatten()), mode='lines',
##              line=dict(width=2, color='darkgreen')), row=5, col=1, secondary_y=True)
##fig.add_trace(go.Scatter(name='Occupancy', x=t, y=occ.flatten(), mode='lines',
##              line=dict(width=1, color='black')), row=6, col=1)
### Set x-axis title
##fig.update_xaxes(title_text="Timestep (-)", row=6, col=1)
### Set y-axes titles
##fig.update_yaxes(title_text="<b>Tair</b> (°C)", range=[10, 24], row=1, col=1)
##fig.update_yaxes(title_text="<b>Tset</b> (°C)", range=[12, 30], row=2, col=1)
##fig.update_yaxes(title_text="<b>PMV</b> (-)", row=3, col=1)
##fig.update_yaxes(title_text="<b>Heat Power</b> (kJ/hr)", row=4, col=1, secondary_y=False)
##fig.update_yaxes(title_text="<b>Heat Energy</b> (kJ)", row=4, col=1, secondary_y=True)
##fig.update_yaxes(title_text="<b>Reward</b> (-)", row=5, col=1, range=[-5, 5], secondary_y=False)
##fig.update_yaxes(title_text="<b>Tot Reward</b> (-)", row=5, col=1, secondary_y=True)
##fig.update_yaxes(title_text="<b>Fraction</b> (-)", row=6, col=1)
##fig.update_xaxes(nticks=50)
##fig.update_layout(template='plotly_white', font=dict(family="Courier New, monospace", size=12),
##                  legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))
##pyo.plot(fig, filename="./results.html")

In [3]:
class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(self, obs_dim: int, size: int, batch_size: int = 32):
        self.obs_dim = obs_dim
        self.obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.next_obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0

    def store(
        self,
        obs: np.ndarray,
        act: np.ndarray, 
        rew: float, 
        next_obs: np.ndarray, 
        done: bool,
    ):
        self.obs_buf[self.ptr] = obs.reshape(self.obs_buf[self.ptr].shape)
        self.next_obs_buf[self.ptr] = next_obs.reshape(self.obs_buf[self.ptr].shape)
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample_batch(self) -> Dict[str, np.ndarray]:
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(obs=self.obs_buf[idxs],
                    next_obs=self.next_obs_buf[idxs],
                    acts=self.acts_buf[idxs],
                    rews=self.rews_buf[idxs],
                    done=self.done_buf[idxs])

    def __len__(self) -> int:
        return self.size

In [4]:
class Network(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        """Initialization."""
        super(Network, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_dim, 128), 
            nn.ReLU(),
            nn.Linear(128, 128), 
            nn.ReLU(), 
            nn.Linear(128, out_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        return self.layers(x)

In [5]:
class DQNAgent:
    """DQN Agent interacting with environment.
    
    Attribute:
        env : (Environment) custom Environment to interact with TRNSYS
        memory (ReplayBuffer): replay memory to store transitions
        batch_size (int): batch size for sampling
        epsilon (float): parameter for epsilon greedy policy
        epsilon_decay (float): step size to decrease epsilon
        max_epsilon (float): max value of epsilon
        min_epsilon (float): min value of epsilon
        target_update (int): period for target model's hard update
        gamma (float): discount factor
        dqn (Network): model to train and select actions
        dqn_target (Network): target model to update
        optimizer (torch.optim): optimizer for training dqn
        transition (list): transition information including 
                           state, action, reward, next_state, done
    """

    def __init__(
        self, 
        env: gym.Env,
        memory_size: int,
        batch_size: int,
        target_update: int,
        epsilon_decay: float,
        max_epsilon: float = 1.0,
        min_epsilon: float = 0.1,
        gamma: float = 0.99,
    ):
        """Initialization.
        
        Args:
            env (gym.Env): custom Environment to interact with TRNSYS
            memory_size (int): length of memory
            batch_size (int): batch size for sampling
            target_update (int): period for target model's hard update
            epsilon_decay (float): step size to decrease epsilon
            lr (float): learning rate
            max_epsilon (float): max value of epsilon
            min_epsilon (float): min value of epsilon
            gamma (float): discount factor
        """
        ## dimensions for the network
        obs_dim = env.observation_dim
        action_dim = env.action_dim
        
        self.env = env
        self.memory = ReplayBuffer(obs_dim, memory_size, batch_size)
        self.batch_size = batch_size
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.target_update = target_update
        self.gamma = gamma
        
        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        print(self.device)

        # networks: dqn, dqn_target
        self.dqn = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()
        
        # optimizer
        self.optimizer = optim.Adam(self.dqn.parameters())

        # transition to store in memory
        self.transition = list()
        
        # mode: train / test
        self.is_test = False

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        # epsilon greedy policy
        if self.epsilon > np.random.random():
            selected_action = np.random.choice(self.env.action_dim,1)[0]
        else:
            selected_action = self.dqn(
                torch.FloatTensor(state.T).to(self.device)
            ).argmax()
            selected_action = selected_action.detach().cpu().numpy()
        
        if not self.is_test:
            self.transition = [state, selected_action]
        
        return selected_action

    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        next_state, reward, done, info = self.env.step(action)

        if not self.is_test:
            self.transition += [reward, next_state, done]
            self.memory.store(*self.transition)
    
        return next_state, reward, done, info

    def update_model(self) -> torch.Tensor:
        """Update the model by gradient descent."""
        samples = self.memory.sample_batch()

        loss = self._compute_dqn_loss(samples)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()
        
    def train(self):
        """Train the agent."""
        self.is_test = False
        
        state = self.env.reset()
        update_cnt = 0
        epsilons = []
        losses = []
        tair = []
        actions = []
        pmv = []
        qheat = []
        rewards = []
        occ = []




        for _ in range(env.numsteps):
            action = self.select_action(state)
            next_state, reward, done,info = self.step(action)


            rewards.append(reward)
            actions.append(env.action_to_temp[action])
            pmv.append(info['pmv'][0])
            d = env.observation_to_dict(next_state)
            tair.append(d["tair_in"][0])
            qheat.append(d["qheat_in"][0])
            occ.append(d["occ_in"][0])
            


            state = next_state

            # if episode ends
            if done:
                state = self.env.reset()

            # if training is ready
            if len(self.memory) >= self.batch_size:
                loss = self.update_model()
                losses.append(loss)
                update_cnt += 1
                
                # linearly decrease epsilon
                self.epsilon = max(
                    self.min_epsilon, self.epsilon - (
                        self.max_epsilon - self.min_epsilon
                    ) * self.epsilon_decay
                )
                epsilons.append(self.epsilon)
                
                # if hard update is needed
                if update_cnt % self.target_update == 0:
                    self._target_hard_update()


        
        ## getting current date for logging reasons
        date = datetime.now()
        temp = list([date.year,date.month,date.day,date.hour,date.minute])
        temp = [str(x) for x in temp]
        time = "_".join(temp)
        RESULT_PATH = "./results/results_{time}"
        os.makedirs(RESULT_PATH,exist_ok=True)



        plot_filename=f"{RESULT_PATH}/results_{time}.html"
        self._plot(epsilons,losses,tair,actions,pmv,qheat,rewards,occ,filename = plot_filename)


        ## saving parameters of environment

        f = open(f"{RESULT_PATH}/env_params_{time}.json","w")
        f.write(json.dumps(env.log_dict,indent=True))
        f.close()


        ## TODO SAVE TRAINING DATA
                
        self.env.close()


    def _plot(
        self, 
        epsilons: List[float],
        losses : List[float],
        tair: List[float],
        actions: List[float],
        pmv : List[float],
        qheat: List[float],
        rewards: List[float],
        occ : List[float],
        plot_filename:str
       ):

        epsilons = np.array(epsilons)
        losses = np.array(losses)
        tair= np.array(tair)
        actions = np.array(actions)
        pmv = np.array(pmv)
        qheat = np.array(qheat)
        rewards = np.array(rewards)
        occ = np.array(occ)


        # Plotting the summary of simulation
        fig = make_subplots(rows=8, cols=1, shared_xaxes=True, vertical_spacing=0.04,
                            specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}],
                                   [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": False}],
                                   [{"secondary_y": True}], [{"secondary_y": True}]])
        
        iterations = len(tair)
        t = np.linspace(0.0, iterations -1, iterations)
        # Add traces
        fig.add_trace(go.Scatter(name='Tair(state)', x=t, y=tair.flatten(), mode='lines', line=dict(width=1, color='cyan')),
                      row=1, col=1)
        fig.add_trace(go.Scatter(name='Tair_avg', x=t, y=pd.Series(tair.flatten()).rolling(window=24).mean(), mode='lines',
                      line=dict(width=2, color='blue')), row=1, col=1)
        fig.add_trace(go.Scatter(name='Tset(action)', x=t, y=actions.flatten(), mode='lines', line=dict(width=1, color='fuchsia')),
                      row=2, col=1)
        fig.add_trace(go.Scatter(name='Tset_avg', x=t, y=pd.Series(actions.flatten()).rolling(window=24).mean(), mode='lines',
                      line=dict(width=2, color='purple')), row=2, col=1)
        fig.add_trace(go.Scatter(name='Pmv', x=t, y=pmv.flatten(), mode='lines', line=dict(width=1, color='gold')),
                      row=3, col=1)
        fig.add_trace(go.Scatter(name='Pmv_avg', x=t, y=pd.Series(pmv.flatten()).rolling(window=24).mean(), mode='lines',
                      line=dict(width=2, color='darkorange')), row=3, col=1)
        fig.add_trace(go.Scatter(name='Heating', x=t, y=qheat.flatten(), mode='lines', line=dict(width=1, color='red')),
                      row=4, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(name='Heating_cumulative', x=t, y=np.cumsum(qheat.flatten()), mode='lines',
                      line=dict(width=2, color='darkred')), row=4, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(name='Reward', x=t, y=rewards.flatten(), mode='lines', line=dict(width=1, color='lime')),
                      row=5, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(name='Reward_cum', x=t, y=np.cumsum(rewards.flatten()), mode='lines',
                      line=dict(width=2, color='darkgreen')), row=5, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(name='Occupancy', x=t, y=occ.flatten(), mode='lines',
                      line=dict(width=1, color='black')), row=6, col=1)
        ## training part

        fig.add_trace(go.Scatter(name='Epsilons', x=t, y=epsilons.flatten(), mode='lines',
                      line=dict(width=1, color='blue')), row=7, col=1)
        fig.add_trace(go.Scatter(name='Training Loss', x=t, y=losses.flatten(), mode='lines',
                      line=dict(width=1, color='darkblue')), row=8, col=1)




        # Set x-axis title
        fig.update_xaxes(title_text="Timestep (-)", row=6, col=1)
        # Set y-axes titles
        fig.update_yaxes(title_text="<b>Tair</b> (°C)", range=[10, 24], row=1, col=1)
        fig.update_yaxes(title_text="<b>Tset</b> (°C)", range=[12, 30], row=2, col=1)
        fig.update_yaxes(title_text="<b>PMV</b> (-)", row=3, col=1)
        fig.update_yaxes(title_text="<b>Heat Power</b> (kJ/hr)", row=4, col=1, secondary_y=False)
        fig.update_yaxes(title_text="<b>Heat Energy</b> (kJ)", row=4, col=1, secondary_y=True)
        fig.update_yaxes(title_text="<b>Reward</b> (-)", row=5, col=1, range=[-5, 5], secondary_y=False)
        fig.update_yaxes(title_text="<b>Tot Reward</b> (-)", row=5, col=1, secondary_y=True)
        fig.update_yaxes(title_text="<b>Fraction</b> (-)", row=6, col=1)
        fig.update_yaxes(title_text="<b>Epsilon</b> (-)", row=7, col=1)
        fig.update_yaxes(title_text="<b>Loss</b> (-)", row=8, col=1)


        fig.update_xaxes(nticks=50)
        fig.update_layout(template='plotly_white', font=dict(family="Courier New, monospace", size=12),
                          legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1))




        
        ## saving plots
        pyo.plot(fig, filename=plot_filename)

        


                

    def _compute_dqn_loss(self, samples: Dict[str, np.ndarray]) -> torch.Tensor:
        """Return dqn loss."""
        device = self.device  # for shortening the following lines
        state = torch.FloatTensor(samples["obs"]).to(device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(device)
        action = torch.LongTensor(samples["acts"].reshape(-1, 1)).to(device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)

        # G_t   = r + gamma * v(s_{t+1})  if state != Terminal
        #       = r                       otherwise
        curr_q_value = self.dqn(state).gather(1, action)
        next_q_value = self.dqn_target(
            next_state
        ).max(dim=1, keepdim=True)[0].detach()
        mask = 1 - done
        target = (reward + self.gamma * next_q_value * mask).to(self.device)

        # calculate dqn loss
        loss = F.smooth_l1_loss(curr_q_value, target)

        return loss

    def _target_hard_update(self):
        """Hard update: target <- local."""
        self.dqn_target.load_state_dict(self.dqn.state_dict())


    # Making a save method to save a trained model
    def save(self, filename, directory):
        torch.save(self.dqn.state_dict(), '%s/%s_dqn.pth' % (directory, filename))
        torch.save(self.dqn_target.state_dict(), '%s/%s_dqn_target.pth' % (directory, filename))

    # Making a load method to load a pre-trained model
    def load(self, filename, directory):
        self.dqn.load_state_dict(torch.load('%s/%s_dqn.pth' % (directory, filename)))
        self.dqn_target.load_state_dict(torch.load('%s/%s_dqn_target.pth' % (directory, filename)))

## Set random seed

In [2]:
seed = 777
env = gym.make('EnergyPlusEnv-v0')

def seed_torch(seed):
    torch.manual_seed(seed)
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

np.random.seed(seed)
seed_torch(seed)
env.seed(seed)

[777]

In [ ]:
columns_keep = ['modelname','numsteps','timestop']

In [ ]:


memory_size = 1000
batch_size = 32
target_update = 100
epsilon_decay = 1 / 2000

agent = DQNAgent(env,memory_size,batch_size,target_update,epsilon_decay)

cuda


In [ ]:
agent.train()

[autoreload of envs.energyplus_env_dir.energyplus_env failed: Traceback (most recent call last):
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "C:\Users\Harold\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: __ini

{'env': <envs.energyplus_env_dir.energyplus_env.EnergyPlusEnv at 0x18a83156610>,
 '_action_space': None,
 '_observation_space': None,
 '_reward_range': None,
 '_metadata': None,
 '_has_reset': True}